In [ ]:
import cv2
import torch
import time
import numpy as np

# Load the YOLOv5m model
try:
    model = torch.hub.load('ultralytics/yolov5', 'yolov5m', pretrained=True)
except Exception as e:
    print(f"Error loading YOLOv5 model: {e}")
    exit()

# Path to the new traffic video
video_path = r'C:\Users\kligh\OneDrive\Desktop\Machine_Learning_Projects()\Traffic_Control_YoloV5\trafficnet_dataset_v1\traffic_detection_prototype2\traffic_video_test_1.mp4'

# Open the video file
cap = cv2.VideoCapture(video_path)

# Check if the video file opened successfully
if not cap.isOpened():
    print(f"Error: Could not open video file: {video_path}")
    exit()

# Get video frame width and height
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the codec and create VideoWriter object
output_path = r'C:\Users\kligh\OneDrive\Desktop\Machine_Learning_Projects()\Traffic_Control_YoloV5\trafficnet_dataset_v1\traffic_detection_prototype2\output_intersection_test_yolov5m_no_filter.mp4'
out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), 30, (frame_width, frame_height))

# Define constants for traffic light control
MIN_GREEN_TIME = 10  # Minimum time to stay in GREEN state
MIN_RED_TIME = 5     # Minimum time to stay in RED state
ORANGE_TIME = 2      # Fixed time for ORANGE state

# Traffic light states
states = ['GREEN', 'ORANGE', 'RED']
current_light = "RED"
light_timer = time.time()  # Timer for the current light

# Classes for vehicles
vehicle_classes = {'car': 2, 'bus': 5, 'truck': 7}  # Class IDs for car, bus, and truck

# Helper function to determine traffic density
def get_traffic_density(vehicle_count):
    if vehicle_count < 5:
        return "low_traffic"
    elif 5 <= vehicle_count < 15:
        return "medium_traffic"
    else:
        return "high_traffic"

# Traffic light logic
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    if not ret:
        print("End of video reached or failed to grab frame.")
        break

    # Perform YOLOv5 inference on the frame
    results = model(frame)

    # Extract detection results
    detections = results.xyxy[0].cpu().numpy()  # Bounding boxes, confidence, class IDs

    # Count vehicles detected in the frame and draw boxes
    vehicle_count = 0
    for *box, conf, class_id in detections:
        class_id = int(class_id)
        if class_id in vehicle_classes.values():  # Only keep vehicle classes
            x1, y1, x2, y2 = map(int, box)
            label = [key for key, value in vehicle_classes.items() if value == class_id][0]
            vehicle_count += 1

            # Draw bounding box and label
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"{label} ({conf:.2f})", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Determine traffic density
    traffic_density = get_traffic_density(vehicle_count)

    # Traffic light decision logic
    elapsed_time = time.time() - light_timer

    if current_light == "GREEN":
        if elapsed_time >= MIN_GREEN_TIME and traffic_density == "low_traffic":
            current_light = "ORANGE"
            light_timer = time.time()
    elif current_light == "ORANGE":
        if elapsed_time >= ORANGE_TIME:
            current_light = "RED"
            light_timer = time.time()
    elif current_light == "RED":
        if elapsed_time >= MIN_RED_TIME and traffic_density == "high_traffic":
            current_light = "GREEN"
            light_timer = time.time()

    # Display the traffic light status on the frame
    cv2.putText(frame, f"Light: {current_light}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, 
                (0, 255, 0) if current_light == "GREEN" else (0, 165, 255) if current_light == "ORANGE" else (0, 0, 255), 2)
    cv2.putText(frame, f"Vehicles: {vehicle_count}", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    # Display the frame
    cv2.imshow('Intersection Traffic Control (YOLOv5m - No Filter)', frame)

    # Save the frame to the output video
    out.write(frame)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and writer
cap.release()
out.release()

# Close OpenCV windows
cv2.destroyAllWindows()


Using cache found in C:\Users\kligh/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-10-19 Python-3.9.18 torch-2.4.1+cpu CPU

Fusing layers... 
YOLOv5m summary: 290 layers, 21172173 parameters, 0 gradients, 48.9 GFLOPs
Adding AutoShape... 
C:\Users\kligh/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\kligh/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\kligh/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\kligh/.cache\torch\hub\ultralytics_yolov5_